In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
filepath = os.getcwd() + '/datasets/'

合并 ab_test_imp 和 ab_test_goal 的数据

In [ ]:
## 读入数据
ab_test_imp = pd.read_csv(filepath + 'section5-ab_test_imp.csv')
ab_test_goal = pd.read_csv(filepath + 'section5-ab_test_goal.csv')
## 合并数据
ab_test_imp = ab_test_imp.merge(ab_test_goal, how='left', on='transaction_id', suffixes=['', '_g'])
ab_test_imp.head()

生成用于标识点击情况的标志位

In [ ]:
ab_test_imp['is_goal'] = np.where(ab_test_imp['user_id_g'].isnull(), 0, 1)
ab_test_imp.head()

统计点击率

In [ ]:
## 计算点击率
ab_test_imp.groupby(by=['test_case']).apply(lambda grp: sum(grp.is_goal)/len(grp.user_id)).rename('ctr').reset_index()

进行卡方检验

In [ ]:
from scipy import stats

# a = list(ab_test_imp.loc[ab_test_imp['test_case'] == 'A', 'is_goal'])
# b = list(ab_test_imp.loc[ab_test_imp['test_case'] == 'B', 'is_goal'])
stats.chisquare( [40592, 3542],[38734, 5056], ddof=0)

In [ ]:
tab = pd.crosstab(ab_test_imp.test_case, ab_test_imp.is_goal)
tab

In [ ]:
stats.chisquare(tab)